# Lấy dữ liệu

In [1]:
%store -r X
%store -r y
%store -r X_test
%store -r y_test
%store -r feature_names

# Thay đổi dữ liệu để phù hợp

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression

# Reshape data
# Data ban đầu X(13166, 20, 128), y(13166)

# Reshape X về dạng 2d => X_2d(263320, 128), giữ nguyên 128 và đó là các đặc trưng
X_2d = X.reshape(-1, X.shape[-1])
# Reshape y thành y_reshaped(263320) bằng cách lặp mỗi phần tử trong y 20 lần
y_reshaped = np.repeat(y, 20)

# Dùng hàm mutual_info_regression
# để đánh giá điểm cho các feature

In [3]:
# Compute information gain for each feature
info_gain = mutual_info_regression(X_2d, y_reshaped)

# Sắp xếp và lấy index của top k phần tử tốt nhất

In [4]:
# Sort the features based on information gain in descending order
sorted_features_indices = np.argsort(info_gain)[::-1]

# Select the top k features (adjust k as needed)
k = 48
selected_features_indices = sorted_features_indices[:k]

In [5]:
selected_X = X[:,:,selected_features_indices]
selected_X_test = X_test[:,:,selected_features_indices]

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Khởi tạo mô hình
model = keras.Sequential()

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(20, k, 1), padding='same'))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5,5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Flatten để làm phẳng đầu ra của lớp trước khi đi vào các lớp fully connected
model.add(layers.Flatten())

# Thêm lớp fully connected (Dense Layer) với 64 units và hàm kích hoạt ReLU
model.add(layers.Dense(64, activation='relu'))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp đầu ra phụ thuộc vào bài toán của bạn
model.add(layers.Dense(5, activation='softmax'))

# In thông tin mô hình
model.summary()

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 20, 48, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 48, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2  (None, 10, 24, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 10, 24, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 24, 64)        51264     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 24, 64)        102464    
                                                     

In [7]:
model.fit(selected_X, y, epochs=10, batch_size=16)

Epoch 1/10


823/823 [==============================] - 33s 38ms/step - loss: 0.9517 - accuracy: 0.6268
Epoch 2/10
823/823 [==============================] - 30s 36ms/step - loss: 0.5500 - accuracy: 0.8089
Epoch 3/10
823/823 [==============================] - 32s 39ms/step - loss: 0.4340 - accuracy: 0.8523
Epoch 4/10
823/823 [==============================] - 32s 39ms/step - loss: 0.3844 - accuracy: 0.8688
Epoch 5/10
823/823 [==============================] - 32s 39ms/step - loss: 0.3669 - accuracy: 0.8707
Epoch 6/10
823/823 [==============================] - 32s 39ms/step - loss: 0.3400 - accuracy: 0.8803
Epoch 7/10
823/823 [==============================] - 32s 39ms/step - loss: 0.3407 - accuracy: 0.8790
Epoch 8/10
823/823 [==============================] - 32s 39ms/step - loss: 0.3214 - accuracy: 0.8878
Epoch 9/10
823/823 [==============================] - 33s 40ms/step - loss: 0.3071 - accuracy: 0.8895
Epoch 10/10
823/823 [==============================] - 33s 40ms/step - loss: 0.2

# k = 48

In [8]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(selected_X_test, y_test)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = model.predict(selected_X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)

412/412 [==============================] - 8s 20ms/step - loss: 0.2079 - accuracy: 0.9218
Độ chính xác trên tập kiểm tra: 0.9218441247940063
412/412 [==============================] - 8s 19ms/step
Ma trận nhầm lẫn:
[[2842    0    0    0    0]
 [   0 1969   13   15   37]
 [   0   31 2567    2  208]
 [   0  376    7 1348    9]
 [   0   30  301    0 3411]]
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2842
           1       0.82      0.97      0.89      2034
           2       0.89      0.91      0.90      2808
           3       0.99      0.77      0.87      1740
           4       0.93      0.91      0.92      3742

    accuracy                           0.92     13166
   macro avg       0.93      0.91      0.92     13166
weighted avg       0.93      0.92      0.92     13166

